# Konfiguracja atomowa a kolor atomu
**Kolory** atomów są często powiązane z **konfiguracją elektronową**, która opisuje rozmieszczenie elektronów wokół jądra atomowego. Konfiguracja elektronowa jest wynikiem podziału elektronów na różne poziomy energetyczne, podpowłok i powłok.

Związki między konfiguracją elektronową a kolorami atomów wynikają z interakcji między elektronami a promieniowaniem elektromagnetycznym. Kiedy atom otrzymuje energię w postaci światła, elektrony na niższych poziomach energetycznych mogą przeskakiwać na wyższe poziomy energetyczne. Gdy elektron powraca do niższego poziomu energetycznego, uwalniana jest energia w postaci światła. Charakterystyka tego emitowanego światła, czyli jego długość fali, determinuje kolor widziany przez nasz wzrok.

Także w teorii powiniśmy móc wykryć różne zależności między konfiguracją atomową, a kolorem atomów. Spróbujmy to zrobić.

## Pobranie bibliotek
Żeby wykonać taką analizę w pythonie potrzebujemy kilku bibliotek. Przede wszystkim *Pandas* pomoże nam w obrabianiu danych, *Colormath* w przeliczaniu parametrów oraz *MatPlotLib* w celu wyświetlenia uzyskanych wyników

In [3]:
import pandas
from colormath.color_objects import sRGBColor, xyYColor, LabColor
from colormath.color_conversions import convert_color
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D

## Dane
Teraz czas na zaczytanie danych. Pobrałem je z [tego repozytorium](https://github.com/andrejewski/periodic-table/blob/master/data.csv) i nieznacznie poprawiłem, ale o tym później. Od razu też zobaczmy jak wyglądają dane, które mamy.

In [ ]:
atomy = pandas.read_csv('data.csv')
print(atomy.head())

     atomicNumber  symbol          name    atomicMass  cpkHexColor   
0               1      H       Hydrogen    1.00794(4)       FFFFFF  \
1               2     He         Helium   4.002602(2)       D9FFFF   
2               3     Li        Lithium      6.941(2)       CC80FF   
3               4     Be      Beryllium   9.012182(3)       C2FF00   
4               5      B          Boron     10.811(7)       FFB5B5   
..            ...     ...           ...           ...          ...   
113           114     Fl      Flerovium         [289]          NaN   
114           115     Mc      Moscovium         [288]          NaN   
115           116     Lv    Livermorium         [293]          NaN   
116           117     Ts     Tennessine         [294]          NaN   
117           118     Og      Oganesson        [294]           NaN   

     electronicConfiguration   electronegativity   atomicRadius  ionRadius   
0                       1s1                 2.20           37.0        NaN  \
1  

Jak widać mamy dostęp od informacji najbardziej podstawowych, jak symbol czy masa, przez potrzebną nam konfigurację i kolor, aż do rozmaitych mniej oczywistych właściwości. Wypiszmy dokładną listę dostępnych właściwości. Zwróć uwagę, że większość nazw ma w sobie spację, co trzeba uwzględnić przy pisaniu kodu, gdybyś też chciał skorzystac z tych danych.

In [5]:
print(atomy.columns.values.tolist())

['atomicNumber', ' symbol', ' name', ' atomicMass', ' cpkHexColor', ' electronicConfiguration', ' electronegativity', ' atomicRadius', ' ionRadius', ' vanDelWaalsRadius', ' ionizationEnergy', ' electronAffinity', ' oxidationStates', ' standardState', ' bondingType', ' meltingPoint', ' boilingPoint', 'density', ' groupBlock', ' yearDiscovered']


Skoro już wiemy dokładnie co chcemy skopiować, to to zróbmy. Stwórzy nową ramkę danych (dataFrame) o nazwie "atomy_kolory", która będzie zawierać informację tylko nam potrzebne, czyli numer, symbol, kolor w hex-ie oraz konfigurację. Odrazu też pozbądźmy się wierszy z pustymi wartościami przy pomocy komendy **dropna**. Największe atomy są wyjątkowo **niestabilne** i udaje się je stworzyć tylko niezwykle krótki czas, pomiar koloru **nie jest przez to możliwy**, stąd puste wiersze któe nas nieinteresują. 

In [6]:
atomy_kolory = atomy[['atomicNumber',  ' symbol', ' cpkHexColor', ' electronicConfiguration']].copy()
atomy_kolory = atomy_kolory.dropna()
print(atomy_kolory)

     atomicNumber  symbol  cpkHexColor  electronicConfiguration
0               1      H        FFFFFF                     1s1 
1               2     He        D9FFFF                     1s2 
2               3     Li        CC80FF                [He] 2s1 
3               4     Be        C2FF00                [He] 2s2 
4               5      B        FFB5B5            [He] 2s2 2p1 
..            ...     ...          ...                      ...
104           105     Db        D1004F       [Rn] 5f14 6d3 7s2 
105           106     Sg        D90045       [Rn] 5f14 6d4 7s2 
106           107     Bh        E00038       [Rn] 5f14 6d5 7s2 
107           108     Hs        E6002E       [Rn] 5f14 6d6 7s2 
108           109     Mt        EB0026       [Rn] 5f14 6d7 7s2 

[109 rows x 4 columns]


Jednak nadal mamy pewien problem, kolor jest zapisany w hex-ie, nie jest to dla nas korzystne. Stwórzmy więc funckję, by zmienić tę wartość na standardowe RGB oraz Lab, obie wartości będą przydatne.

In [8]:
def kolor_konversion(hex):
    ColorHex = hex
    R = ColorHex[0:2]
    G = ColorHex[2:4]
    B = ColorHex[4:6]

    B = int(B, base=16)
    G = int(G, base=16)
    R = int(R, base=16)

    kolor_sRGB = sRGBColor(R, G, B, is_upscaled = True)
    kolor_Lab = convert_color(kolor_sRGB, LabColor, target_illuminant='d50')
    Lab = kolor_Lab.get_value_tuple()
    return((kolor_sRGB.rgb_r, kolor_sRGB.rgb_g, kolor_sRGB.rgb_b), Lab)

Teraz dodajmy dwie kolumny z wartościami RGB oraz Lab. Dodam tutaj, że musiałem poprawić dwie wartości, które były niepoprawne. Palladium poprawiłem z 6985 do b1b1b1 a Erbium z 0 na 00FF9C.

In [9]:
HexKolorDF = atomy_kolory[' cpkHexColor']
wektorKolor = []
for i in HexKolorDF:
    wektorKolor.append(kolor_konversion(i))

RGB = []
Lab = []
for i in range(len(wektorKolor)):
    RGB.append(wektorKolor[i][0])
    Lab.append(wektorKolor[i][1])
atomy_kolory['RGB_color'] = RGB
atomy_kolory['LAB_color'] = Lab
print(atomy_kolory.head())

   atomicNumber  symbol  cpkHexColor  electronicConfiguration   
0             1      H        FFFFFF                     1s1   \
1             2     He        D9FFFF                     1s2    
2             3     Li        CC80FF                [He] 2s1    
3             4     Be        C2FF00                [He] 2s2    
4             5      B        FFB5B5            [He] 2s2 2p1    

                                       RGB_color   
0                                (1.0, 1.0, 1.0)  \
1                 (0.8509803921568627, 1.0, 1.0)   
2                 (0.8, 0.5019607843137255, 1.0)   
3                 (0.7607843137254902, 1.0, 0.0)   
4  (1.0, 0.7098039215686275, 0.7098039215686275)   

                                           LAB_color  
0  (99.9998893565855, -0.0012758568349036103, -0....  
1  (97.3041556015011, -12.609871764856228, -4.311...  
2  (65.72658844657771, 46.60212363651001, -52.273...  
3  (93.31619546771647, -39.55956206798672, 88.003...  
4  (80.8363466163351,

## Wyświetlanie danych
Czas na wyświetlenie naszych wyników i próbę znalezienia w nich jakiegoś sensu. Punkty wyświetlać będziemy w przestrzeni **Lab**, a każdy punkt będzie odpowiednio pokolorowany zgodnie z jego parametrami **RGB**.

In [ ]:
%matplotlib notebook

Lab = atomy_kolory["LAB_color"]
RGB = atomy_kolory['RGB_color']
config = atomy_kolory[' electronicConfiguration']
atom = atomy_kolory[' symbol']
x = []
y = []
z = []
color = []
podpis = []
for i in range(len(Lab)):
    x.append(Lab[i][1])
    y.append(Lab[i][2])
    z.append(Lab[i][0])
    color.append(RGB[i])
    podpis.append(atom[i])

fig = plt.figure()
ax = Axes3D(fig)

#fig = plt.figure(figsize=(16,14))
#ax = plt.axes(projection='3d')

# Plot the data points as a scatter plot
ax.scatter3D(x, y, z, c=color)
for i in range(len(podpis)):
    ax.text(x = x[i],y = y[i],z = z[i], s = podpis[i], zdir = None)

# Set labels for the axes
ax.set_xlabel('a')
ax.set_ylabel('b')
ax.set_zlabel('L')

# Set a title for the plot
ax.set_title('3D Scatter Plot')

# Display the plot
plt.show()